## Qualitative coding using openai

- Prompt 1 (Zero shot):input the entire codebook, including a table that has parent code, parent code description, child code, and child code description; use the GPT to label both child and parent code for each document; if non child code work, just label the parent code.

- Prompt 2 (Chain of thoughts):  Step 1: label all parent codes with reasoning; Step 2; label child code within each parent code. if none of the child code work, just keep the parent code.

In [3]:
# install openai library
#pip install openai
!pip install langchain==0.0.316
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.3/567.3 kB 15.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
import openai
from openai import OpenAI
import os
import ast
import time
import numpy as np
import json
from IPython.display import clear_output
from tqdm import tqdm
import re

In [2]:
tqdm.pandas()

In [3]:
# Set work directory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files
uploaded = files.upload()

Saving sample_interview_data.csv to sample_interview_data.csv


## Load data

In [5]:
# Import text data
# Use interview_data.csv here

df = pd.read_csv('sample_interview_data.csv', index_col = 0)

In [6]:
# Example text for GPT to label
df['documents'].head()

,documents
roles,
Educator,"So speaking at a local district level, we have..."
Educator,"I know that my field, the area of my expertise..."
Educator,"In terms of local policies, something that can..."
Educator,I think you were talking before about this dat...
Educator,As a educator who works with students directly...


In [7]:
# Function to preprocess text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    #text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.strip()  # Trim spaces
    return text

# Apply preprocessing to the documents column
df['documents'] = df['documents'].astype(str).apply(preprocess_text)

# LLM API Integration

In [8]:
import getpass

In [9]:
# You can set up .env file to restore the api key
openai.api_key = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


## Open coding

### Function to extract themes from the each row

In [10]:
from pydantic import BaseModel

In [12]:
class QualCode(BaseModel):
  name: str
  description: str

class ListOfCodes(BaseModel):
  codes: list[QualCode]


In [13]:
class QualCode(BaseModel):
  name: str
  description: str

class ListOfCodes(BaseModel):
  codes: list[QualCode]

def get_open_codes(text):
    prompt = f"""
    You are an expert qualitative researcher conducting open coding.
    You are provided with text from a transcript of an interview with an education policy stakeholder.

    Text: "{text}"

    Provide your response in JSON format with two keys: "codes" (a list of short codes) and "descriptions" (a corresponding description for each code).

    Example Response:
    {{
        "codes": ["Student Engagement", "Technology Use"],
        "descriptions": ["The text describes how students engage with the content", "The role of technology in enhancing learning"]
    }}
    """

    client = OpenAI(api_key=openai.api_key)

    try:
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            response_format=ListOfCodes,
        )

        print("original response: ", f"{completion}")

        # Parse LLM output

        # Extract parsed structured data
        parsed_response = completion.choices[0].message.parsed.codes


        print("open coding result: ", parsed_response)

        # If the model refuses to respond
        # if parsed_response.refusal:
        #     print(parsed_response.refusal)
        #     return None


        # Extract names and descriptions separately
        names = [code.name for code in parsed_response]
        descriptions = [code.description for code in parsed_response]

        return (parsed_response, names, descriptions)


    except Exception as e:
        print(f"Error processing text: {text}\nError: {str(e)}")
        return {"codes": [], "descriptions": []}


In [15]:
# Select a subset of text to test prompt
#random_rows = df.sample(n=50, random_state=2023)
t = df.iloc[8:10, ]

In [16]:
# Provide one row at a time
# Apply open coding

t[['Open_coding', 'Codes', 'Descriptions']] = t['documents'].apply(get_open_codes).apply(pd.Series)


# Extract codes and descriptions into separate columns
# t['Open_Coding'] = open_coding
# t['Codes'] = codes
# t['Descriptions'] = descriptions

original response:  ParsedChatCompletion[ListOfCodes](id='chatcmpl-BBpPQWeJB2o6xZ5APVdwQ55v8qgyY', choices=[ParsedChoice[ListOfCodes](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[ListOfCodes](content='{"codes":[{"name":"Equity Policy","description":"The introduction and approval of the first equity policy at the district level."},{"name":"Leadership Change","description":"Change in leadership with the departure of the full-time director of equity and the appointment of a part-time replacement."},{"name":"District Initiatives","description":"Efforts and initiatives taken by the district to promote equity."},{"name":"Superintendent Influence","description":"The role of the new superintendent in driving the approval of the equity policy."}]}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=ListOfCodes(codes=[QualCode(name='Equity Policy', description='The introduction and approval of the first equity policy at the

ValueError: Columns must be same length as key

In [33]:
test_material = "{\n    \"codes\": [\"Dual Language Initiatives\", \"Language Skills Development\"],\n    \"descriptions\": [\"The stakeholder supports Superintendent Reykdal's dual language initiatives\", \"Emphasis on ensuring students have the necessary language skills for their future goals\"]\n}".strip()


In [48]:
json.loads(test_material)

{'codes': ['Dual Language Initiatives', 'Language Skills Development'],
 'descriptions': ["The stakeholder supports Superintendent Reykdal's dual language initiatives",
  'Emphasis on ensuring students have the necessary language skills for their future goals']}

In [49]:
t

,#doc,documents,Open_coding,Codes,Descriptions
roles,,,,,
Educator,18.0,in terms of what is the quality that we are ha...,"[name='Teacher Dependence' description=""The qu...","[Teacher Dependence, Diverse Teacher Backgroun...",[The quality of education is largely dependent...
Educator,19.0,i know i had some conversations last year with...,[name='Dual Language Support' description='The...,"[Dual Language Support, Student Autonomy, Engl...",[The text highlights the importance of support...


In [52]:
# Refine and label emerging themes
# Pattern Recognition
# Validation Techniques
t.head()

,#doc,documents,Open_Coding,Codes,Descriptions
roles,,,,,
Educator,18.0,in terms of what is the quality that we are ha...,"{'codes': [{'name': 'Quality Variability', 'de...","[{'name': 'Quality Variability', 'description'...",[]
Educator,19.0,i know i had some conversations last year with...,{'codes': [{'name': 'Dual Language Initiatives...,"[{'name': 'Dual Language Initiatives', 'descri...",[]


In [19]:
# Flatten and aggregate into a codebook
pre_codebook1 = []
for _, row in t.iterrows():
    for code, desc in zip(row['Codes'], row['Descriptions']):
        pre_codebook1.append({"Code": code, "Description": desc})


In [20]:
pre_codebook1

[{'Code': 'Teacher Quality',
  'Description': 'The text emphasizes the importance of teacher quality in the region'},
 {'Code': 'Curriculum Flexibility',
  'Description': 'The stakeholder supports a flexible curriculum approach that allows for diverse teacher backgrounds'},
 {'Code': 'Common Understanding',
  'Description': 'The stakeholder highlights the need for a common understanding among educators and administrators'},
 {'Code': 'Dual Language Initiatives',
  'Description': "The stakeholder is in support of Superintendent Reykdal's dual language initiatives"},
 {'Code': 'Language Development',
  'Description': "The importance of meaningful language development in students' native and heritage languages"}]

#### Function to extract collective themes from the entire dataset

In [ ]:
def extract_themes(text_list):
    combined_text = " ".join(text_list)  # Merge all texts into a single document

    prompt = f"""
    You are an expert qualitative researcher performing open coding.
    You are provided with interview transcripts with education policy stakeholders.

    Text: "{combined_text}"

    Provide your response in JSON format:
    {{
        "Codes": ["Theme 1", "Theme 2", "Theme 3"],
        "Descriptions": ["Brief description of Theme 1", "Description of Theme 2", "Description of Theme 3"]
    }}
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )

        # Parse the JSON output
        content = response['choices'][0]['message']['content'].strip()
        return json.loads(content)

    except Exception as e:
        print(f"Error: {str(e)}")
        return {"Codes": [], "Descriptions": []}  # Return empty structure on failure



In [ ]:
# Load text data
documents_list = t['documents'].tolist()

# Step 1: Extract collective themes
pre_codebook = extract_themes(documents_list)
themes, descriptions = pre_codebook["Codes"], pre_codebook["Descriptions"]

In [ ]:
pre_codebook

{'Codes': ['Teacher Quality',
  'Dual Language Initiatives',
  'Language Development'],
 'Descriptions': ['Discussion on the importance of teacher quality in education',
  "Support for Superintendent Reykdal's dual language initiatives",
  'Focus on meaningful language development for students in their native and heritage languages']}

In [ ]:
# Function to assign themes to each text entry
def assign_themes(text_list, themes, descriptions):
    prompt = f"""
    Given the following overarching themes:
    {json.dumps({"themes": themes, "descriptions": descriptions}, indent=2)}

    Now classify each text into one or more of these themes.

    Texts: {json.dumps(text_list, indent=2)}

    Provide a response in JSON format as a list where each element contains:
    {{
        "assigned_themes": ["Theme 1", "Theme 2"]
    }}
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )

        # Parse the JSON output
        content = response['choices'][0]['message']['content'].strip()
        return json.loads(content)

    except Exception as e:
        print(f"Error: {str(e)}")
        return [{"assigned_themes": []} for _ in text_list]  # Return empty structure on failure

# Step 2: Assign themes to individual texts
assigned_theme_results = assign_themes(documents_list, themes, descriptions)

# Convert results to DataFrame
t['Assigned_Themes'] = [result["assigned_themes"] for result in assigned_theme_results]


<ipython-input-57-f89eda91d7de>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['Assigned_Themes'] = [result["assigned_themes"] for result in assigned_theme_results]


In [ ]:
t['Assigned_Themes']

,Assigned_Themes
roles,
Educator,"[Teacher Quality, Language Development]"
Educator,"[Dual Language Initiatives, Language Development]"


## Codebook-based coding

### Load codebook

In [ ]:
# Import codebook if applicable
# If you are open coding using LLM, skip this
# Use codebook.csv here
codebook = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Ballmer/data/codebook.csv')

In [ ]:
# The codebook contains parent and child codes from which GPT selects.
codebook.head()

,Topic,Parent,Child,Child_description,Agreed rating,Min's Original Coherence rating,Key words
0,23.0,"Culture, climate and environment",Trauma at home,Struggling home and family experiences of chil...,4.0,4.0,"bad, kid, children, home, school, famili, grad..."
1,14.0,"Culture, climate and environment",Anti-racism,"Talking about whiteness, success, and anti-racism",3.0,3.0,"pull, white, child, success, stay, built, job,..."
2,NaN,Curriculum and instruction,Instructional programs,"AP/IB courses, college classes/credits in high...",NaN,NaN,"Equity, Students, School, District, Policy, Da..."
3,13.0,Curriculum and instruction,Curriculum development and instructional delivery,School curriculum development and instructiona...,3.0,3.0,"onlin, teach, taught, teacher, high, learn, cl..."
4,11.0,"Data, evidence, and accountability","Data access, analysis, reporting, use, quality...","Data collection, access, analysis, and use to ...",4.0,4.0,"dashboard, data, inform, assumess, collect, di..."


### *load functions*

In [ ]:
def label_topic(doc_in):
    """
    Labels the topic of a given document using OpenAI's GPT model.

    Args:
    doc_in (str): The document or text snippet for which the topic needs to be determined.

    Returns:
    current_response: The labeled topic as determined by the GPT model or an error message.
    """

    # Create a prompt using the document input.
    prompt = create_prompt(doc_in)

    # Send the generated prompt to the OpenAI API and get a response.
    response = send_to_api(prompt)

    # Check if the response from the API is not an error.
    if response != "InvalidRequestError":
        current_response = response.choices[0].message.content
    else:
        current_response = response

    return current_response

In [ ]:
def send_to_api(prompt):
    """
    This function sends a given prompt to OpenAI's API and retrieves the response.

    Args:
    prompt (str): A string containing the user's input that needs to be processed by the GPT model.

    Returns:
    result: The response from OpenAI's API or an error message if the request fails.
    """

    # Initialize result to None, this will store the API response
    result = None

    # A loop to attempt the API call multiple times if necessary
    i = 1
    while result is None:
        try:
            # Making a request to OpenAI's ChatCompletion API
            result = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",  # Specifies the GPT model to use
                messages=[
                    {"role": "user", "content": prompt}  # The prompt or message for the GPT model
                ],
                temperature=0.5  # Sets the creativity of the response. Lower is more deterministic.
            )

        except openai.InvalidRequestError:
            # If there is an invalid request, store an error message
            result = "InvalidRequestError"
        except:
            # For other errors, wait for 1.5 seconds before retrying
            time.sleep(1.5)
            pass

    # Return the API response or error message
    return result

send_to_api: This function is a robust way to interact with the OpenAI API, handling potential errors and retrying requests. The `temperature` parameter can be adjusted to alter the nature of the generated responses, with lower values leading to more predictable and conservative outputs.

### Prompt 1 (Zero shot):input the entire codebook, including a table that has parent code, parent code description, child code, and child code description; use the GPT to label both child and parent code for each document; if non child code work, just label the parent code.

#### Prompt archive

In [ ]:
# prompt 0-0
prompt_base = f"""
You are provided with a paragraph from qualitative interviews and a codebook that contains parent and child codes for qualitative analysis. Each child code is associated with a description in the codebook.
Please assist in performing qualitative coding for the paragraph from qualitative interviews based on the provided codebook. The codebook is in CSV format and contains four useful columns for this task: 'Parent', 'Child', 'Child_description' and 'Key words'.

Here's how you should proceed:

1. Review the codebook:
   - The 'Parent' is a categorical label for the broader theme or category.
   - The 'Child' is a categorical label for the detailed theme.
   - The 'Child_description' provides a detailed description of what each child code in corresponding 'Child' represents.
   - The 'Key words' provides high frequent words that are mentioned in a child code in 'Child'.

2. For the paragraph from the qualitative interviews, assign a code from the codebook. Use the following steps:
   a. Read the paragraph.
   b. Refer to the codebook to find the most appropriate 'Parent', 'Child', and 'Child_description' for this paragraph.
   c. Provide the 'Parent', 'Child', and 'Child_description' as labels for this paragraph.

Here is a snippet from the codebook:
{codebook}

--- Paragraph to Label ---
[TEXTGOHERE]

Your task is to label the paragraph from the qualitative interviews based on the codes defined in the codebook. For each paragraph, identify the relevant parent code and child code from the codebook and provide a brief reasoning of why you selected that child code.
Please ensure that the labels you assign accurately represent the content of the paragraph. If you are uncertain or if a paragraph does not fit any code from the codebook, please return 'None'.

Format your response as a JSON object with 3 keys where
“Parent code”, “Child code”, and “Reasoning” as the keys.
"""

In [ ]:
# prompt 1
prompt_base = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to performing qualitative coding for the Paragraph using the Codebook provided below. The Codebook contains four useful columns for this task: 'Parent', 'Child', 'Child_description' and 'Key words'.

First, review the Codebook:
   - 'Parent' column includes categorical labels for broader themes or categories.
   - 'Child' column includes categorical labels for detailed themes, nested within corresponding 'Parent' categories.
   - 'Child_description' column provides detailed descriptions of what corresponding 'Child' represents.
   - 'Key words' column provides highly frequent words that are relevant to corresponding 'Child' categories.

Second, using the Codebook, identify the top three most prevalient themes in each Paragraph by following the following steps:
   a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system;
   b. Refer to the Codebook to identify three most salient themes emerge from the paragraph using 'Child' based on corresponding 'Child_description'; only when you cannot identify any appropriate child code, use the Parent. Show your reasoning behind your classifications;
   c. If a Paragraph does not fit any Child and Parent category from the Codebook, please return 'None'.

Here is the Codebook:
{codebook}

--- Paragraph ---
[TEXTGOHERE]

Format your response as a JSON object with 3 keys where
“Theme 1”, “Theme 2”, “Theme 3” and “Reasoning” as the keys.
"""

In [ ]:
# prompt 1-0 remove keywords, up to 3, simplify steps
prompt_base = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to use the Codebook in CSV format to code Paragraph from the interview transcript.

First, review and understand the Codebook:
   - 'Parent' column includes categorical labels for broader themes.
   - 'Child' column includes categorical labels for subthemes, nested within corresponding 'Parent' categories.
   - 'Child_description' column provides detailed descriptions of subcategories acco 'Child' represents.

Second, using the Codebook, identify up to three the most salient themes in each Paragraph, which cover the most noticeable, central, important idea conveyed in the Paragraph, by following the following steps:
   a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system.
   b. Identify up to three most salient themes from the 'Child' categories, based on 'Child_description'. ONLY if you could not identify an appropriate 'Child' categories, you can use the 'Parent' categories.
   c. If a Paragraph does not fit any Child or Parent categories from the Codebook, return 'None'.
   d. Review the identified themes, if they do not reflect the most salient themes of the Paragraph, repeat step a-c for 3 time maximum.
   e. Provide the identified themes as labels for this Paragraph and show your reasoning behind your classifications.

Here is the Codebook:
{codebook}

--- Paragraph ---
[TEXTGOHERE]

Format your response as a JSON object with 4 keys where
“Theme 1”, “Theme 2”, “Theme 3” and “Reasoning” as the keys.
"""

In [ ]:
# prompt 1-0 remove keywords, up to 3, simplify steps
prompt_base1 = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to use the Codebook in CSV format to code Paragraph from the interview transcript.

First, review the Codebook:
   - 'Parent' column includes categorical labels for broader themes.
   - 'Child' column includes categorical labels for detailed themes, nested within corresponding 'Parent' categories.
   - 'Child_description' column provides detailed descriptions of what each child category in 'Child' represents.

Second, using the Codebook, identify three most salient themes in each Paragraph, which cover the most noticeable, central, important idea conveyed in the Paragraph, by following the following steps:
   a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system.
   b. Identify three most salient themes from the 'Child' categories, based on 'Child_description'. ONLY if you could not identify an appropriate 'Child' categories, you can use the 'Parent' categories.
   c. Review the three identified themes, if they do not reflect the most salient themes of the Paragraph, repeat step a-b for 3 time maximum. If a Paragraph does not fit any Child or Parent categories from the Codebook, return 'None'.
   d. Provide the identified themes as labels for this Paragraph and show your reasoning behind your classifications.

Here is the Codebook:
{codebook}

--- Paragraph ---
[TEXTGOHERE]

Format your response as a JSON object with 4 keys where
“Theme 1”, “Theme 2”, “Theme 3” and “Reasoning” as the keys.
"""

In [ ]:
# v02
prompt_base = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to use the Codebook in CSV format to code Paragraph from the interview transcript.

First, review the Codebook:
- 'Parent' column includes categorical labels for broader themes.
- 'Child' column includes categorical labels for detailed themes, nested within corresponding 'Parent' categories.
- 'Child_description' column provides detailed descriptions of what each child category in 'Child' represents.
- 'Key words' column provides high-frequency words that are relevant to corresponding 'Child' categories.

Second, using the Codebook, identify three most salient themes in each Paragraph, which cover the most noticeable, central, important idea conveyed in the Paragraph, by following the following steps:
a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system;
b. Identify three of the most salient themes from the 'Child' categories, based on 'Child_description'; you can also refer to 'Key words'; ONLY if you could not identify an appropriate 'Child', you can use the 'Parent' categories. Provide the three identified themes as labels for this Paragraph and show your reasoning behind your classifications;
c. Please ensure that the themes you identified accurately represent the content of the Paragraph; if a Paragraph does not fit any Child or Parent categories from the Codebook, return 'None'.

Here is the Codebook:
{codebook}

--- Paragraph ---
[TEXTGOHERE]

Format your response as a JSON object with 4 keys where
“Theme 1”, “Theme 2”, “Theme 3” and “Reasoning” as the keys.
"""


#### Prompt in use

In [ ]:
# v03
prompt_base = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to use the Codebook in CSV format to code Paragraph from the interview transcript.


First, review and learn the Codebook:
- 'Parent' column includes short labels for broader themes or categories of ideas.
- 'Child' column includes labels for subcategories of corresponding 'Parent' categories.
- 'Child_description' column provides narrative descriptions of 'Child' categories.
- 'Key words' column provides high-frequency words that are relevant to corresponding 'Child' categories.


Second, using the Codebook, identify the top three most salient themes in each Paragraph by following the following steps:
a. Read the Paragraph and understand the meaning in the context of the Washington State K-12 public school system.
b. Identify the three most salient themes using the 'Child' categories based on 'Child_description'. You can also refer to 'Key words'. Only when you cannot identify an appropriate Child category, you can label the Paragraph by using the Parent categories.
c. Provide the three identified themes as labels for this Paragraph and show your reasoning behind your classifications.
c. Review and ensure that the themes you identified accurately represent the content of the Paragraph. If a Paragraph does not fit any Child or Parent category from the Codebook, return 'None'.

Here is a snippet from the Codebook:
{codebook}

--- Paragraph ---
[TEXTGOHERE]


Format your response as a JSON object with 4 keys where
“Theme 1”, “Theme 2”, “Theme 3” and “Reasoning” as the keys.
"""


In [ ]:
# Select a subset of text to test prompt
#random_rows = df.sample(n=50, random_state=2023)
t = df.iloc[8:10, ]

In [ ]:
def create_prompt(doc_in):
    return prompt_base.replace('TEXTGOHERE', doc_in)

In [ ]:
output = []
for index, row in tqdm(t.iterrows(), total = t.shape[0]):
  codebook=codebook[['Parent', 'Child', 'Child_description', 'Key words']]
  doc_in = row.documents # text column
  output_current = label_topic(doc_in)
  output.append(output_current)

100%|██████████| 2/2 [00:03<00:00,  1.97s/it]


In [ ]:
print(output[0])

{
    "Theme 1": "Curriculum and instruction",
    "Theme 2": "Governance, leadership, and community partnership",
    "Theme 3": "Staffing resources",
    "Reasoning": "The paragraph discusses the quality of education in the region being teacher-dependent, indicating a focus on curriculum and instruction. It also highlights the importance of common understanding among teachers, administration, and state-level officials, pointing towards governance, leadership, and community partnership. Additionally, the mention of teachers with diverse backgrounds and experiences suggests a need for adequate staffing resources to support the varied needs of educators."
}


### Prompt 2 (Chain of thought): Step 1: label all parent codes with reasoning; Step 2; label child code within each parent code. if none of the child code work, just keep the parent code.

#### Prompt archive

In [ ]:
prompt_base2 = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to use the Codebook in CSV format to code Paragraph from the interview transcript. The Codebook contains four columns: 'Parent', 'Child', 'Child_description' and 'Key words'.

Here's how you should proceed:

1. Review parent code in the Codebook:
   - 'Parent' column includes categorical labels for broader themes or categories.

2. Using the Codebook, identify up to three the most salient themes in each Paragraph, which should cover the most noticeable, central, important idea conveyed in the Paragraph, by following the following steps:
   a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system;
   b. Identify up to three the most salient themes from the Parent categories.
   c. Provide identified 'Parent' as label for this Paragraph.

3. Review child code in the Codebook:
   - 'Child' column includes categorical labels for detailed themes, nested within corresponding 'Parent' categories.
   - 'Child_description' column provides detailed descriptions of what each child category in 'Child' represents.
   - 'Key words' column provides highly frequent words that are relevant to corresponding 'Child' categories.

4. Using the Codebook, within the an identified 'Parent' category, identify the most salient 'Child' categories. Use the following steps:
   a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system;
   b. Refer to the Codebook, for each identified 'Parent' category, find the most appropriate subcategories from 'Child' for this Paragraph based on 'Child_description'. You can also refer to 'Key words'.
   c. Please ensure that the 'Child' you assign accurately represent the content of the Paragraph. If you are uncertain or if a Paragraph does not fit any 'Child' categories from the Codebook, please return 'None'.
   d. Review all identified 'Parent' and 'Child' pairs, select three pairs that represent the most noticeable, central, important idea conveyed in the Paragraph.
   e. Provide identified three pairs of 'Parent' and their corresponding 'Child' as labels for this paragraph.

Here is the Codebook:
{codebook}

--- Paragraph to Label ---
[[[TEXTGOHERE]]]

Format your response as a JSON object with 7 keys where
'Parent 1', 'Child 1', 'Parent 2', 'Child 2', 'Parent3', 'Child 3', and 'Reasoning' as the keys.
"""

In [ ]:
prompt_base2_1 = f"""
You are a policy researcher. You are provided with a Paragraph from a transcript of an interview with an education policy stakeholder.
Your task is to use the Codebook in CSV format to code Paragraph from the interview transcript. The Codebook contains four columns: 'Parent', 'Child', 'Child_description' and 'Key words'.

Here's how you should proceed:

1. Using the Codebook, identify up to three the most salient themes in each Paragraph, which should cover the most noticeable, central, important idea conveyed in the Paragraph, by following the following steps:
a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system;
b. Review Parent Code in the Codebook: ‘Parent’ column includes categorical labels for broader themes
c. Identify up to three the most salient themes from the Parent categories.
d. Provide identified 'Parent' as label for this Paragraph.

2. Review child code in the Codebook:
- 'Child' column includes categorical labels for detailed themes, nested within corresponding 'Parent' categories.
- 'Child_description' column provides detailed descriptions of what each child category in 'Child' represents.
- 'Key words' column provides highly frequent words that are relevant to corresponding 'Child' categories.

3. Using the Codebook, within the an identified 'Parent' category, identify the most salient 'Child' categories. Use the following steps:
a. Read the Paragraph and understand the meaning in the context of Washington State K-12 public school system;
b. Refer to the Codebook, for each identified 'Parent' category, find the most appropriate subcategories from 'Child' for this Paragraph based on 'Child_description'. You can also refer to 'Key words'.
c. Please ensure that the 'Child' you assign accurately represents the content of the Paragraph. If you are uncertain or if a Paragraph does not fit any 'Child' categories from the Codebook, please return 'None'.
d. Review all identified 'Parent' and 'Child' pairs, select three pairs that represent the most noticeable, central, important idea conveyed in the Paragraph.
e. Provide identified three 'Parent' and their corresponding 'Child' as labels for this paragraph.

Here is the Codebook:
{codebook}

--- Paragraph to Label ---
[[[TEXTGOHERE]]]

Format your response as a JSON object with 7 keys where
'Parent 1', 'Child 1', 'Parent 2', 'Child 2', 'Parent3', 'Child 3', and 'Reasoning' as the keys.
"""


#### Prompt in use

In [ ]:
prompt_base2 = f"""
Task: As a policy researcher, you’ve been provided with a paragraph extracted from an interview with an education policy stakeholder. Utilize the provided Codebook (in CSV format) to code the paragraph. The Codebook comprises four columns: ‘Parent’, ‘Child’, ‘Child_description’, and ‘Key words’.

Steps:
1. Identify Salient Themes:
Understand the paragraph’s content within the context of the Washington State K-12 public school system.
Refer to the ‘Parent’ column in the Codebook for broader thematic categories.
Pinpoint up to three salient themes from these ‘Parent’ categories.
These themes should highlight the most significant ideas in the paragraph.
Label the paragraph with the chosen ‘Parent’ themes.

2. Dive into Child Themes:
The ‘Child’ column in the Codebook lists detailed thematic subcategories, which fall under the broader ‘Parent’ categories.
The ‘Child_description’ elaborates on the ‘Child’ categories, and the ‘Key words’ column lists pertinent terms for each ‘Child’ category.

3. Associate with Child Categories:
Revisit the paragraph, keeping the Washington State K-12 public school system context in mind.
For each previously identified ‘Parent’ theme, pinpoint the apt ‘Child’ subcategories from the Codebook. The ‘Child_description’ and ‘Key words’ columns can aid your decision.
Ensure the ‘Child’ categories align with the paragraph’s content. If there’s no fit or you’re uncertain, label it as ‘None’.
From your identified ‘Parent’ and ‘Child’ pairs, pick the top three pairs that encapsulate the paragraph’s central ideas.
Label the paragraph with these three ‘Parent’ and corresponding ‘Child’ pairs.

Codebook:
{codebook}

Paragraph for Analysis:
[[[TEXTGOHERE]]]

Response Format:
Frame your answer as a JSON object containing the keys: ‘Parent 1’, ‘Child 1’, ‘Parent 2’, ‘Child 2’, ‘Parent 3’, ‘Child 3’, and ‘Reasoning’.
"""

In [ ]:
def create_prompt(doc_in):
    return prompt_base2.replace('TEXTGOHERE', doc_in)

In [ ]:
t = df.iloc[8:10, ]

In [ ]:
output2 = []
for index, row in tqdm(t.iterrows(), total = t.shape[0]):
        doc_in = row.documents
        output_current = label_topic(doc_in)
        output2.append(output_current)

100%|██████████| 2/2 [00:18<00:00,  9.31s/it]


In [ ]:
# cot prompt results
print(output2[0])

{
  "Parent 1": "Staffing resources",
  "Child 1": "Mentoring, coaching, and teacher learning",
  "Parent 2": "Curriculum and instruction",
  "Child 2": "Curriculum development and instructional delivery",
  "Parent 3": "System supports and interventions",
  "Child 3": "School system support and improvement",
  "Reasoning": "The paragraph primarily discusses the topic of new teachers and their experience in the classroom, specifically their lack of prior student teaching experience. This falls under the 'Parent' category of 'Staffing resources', and more specifically the 'Child' category 'Mentoring, coaching, and teacher learning' as it touches on the need for teaching experience before becoming educators. The 'Parent' category of 'Curriculum and instruction' and 'Child' category 'Curriculum development and instructional delivery' are also relevant as the paragraph implies the need for practical experience in delivering instruction. Finally, 'System supports and interventions' and 'Sch

In [ ]:
# zero-shot prompt results
print(output[0])

{
  "Theme 1": "Mentoring, coaching, and teacher learning",
  "Theme 2": "Instructional programs",
  "Theme 3": "Curriculum development and instructional delivery",
  "Reasoning": "The paragraph discusses the challenges faced by new teachers who didn't have the opportunity to gain student teaching experience. This relates to the 'Mentoring, coaching, and teacher learning' theme because it's about providing support and learning opportunities for new teachers. The mention of teachers being inside a classroom for the first time also touches on 'Instructional programs' and 'Curriculum development and instructional delivery' as they are expected to deliver curriculum and instruction for the first time."
}


## Sentiment Analysis